In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import pandas as pd
import pickle
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
gbr_params = {'n_estimators': 1000,
          'max_depth': 5,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

In [3]:
states_df = pd.read_csv("../../data/lookup_data/states.csv")
states_df.head(50)

,state_code,state_name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California
5,CO,Colorado
6,CT,Connecticut
7,DE,Delaware
8,FL,Florida
9,GA,Georgia


In [4]:
def buildModel(df, state_code, metric):
    
    print(df.head())
    
    # Filter for state
    df = df[df['state_code'] == state_code]
    
    #Add remove features
    df.drop(["state_code"],inplace=True,axis=1)
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['day'] = pd.DatetimeIndex(df['date']).day
    
    # Create Training and Test Split
    X_train, X_test, y_train, y_test = train_test_split(df[['year','month','day']].values, df[['value']].values.flatten(), random_state=42, test_size=0.1)

    # Standardize the dataset
    sc = StandardScaler()
    X_train_std = sc.fit_transform(X_train)
    scaler_file = f"modelfiles/{metric}-{state_code}-scaler.sav"
    pickle.dump(sc, open(scaler_file,'wb'))
    
    X_test_std = sc.transform(X_test)

    # Create an instance of gradient boosting regressor
    gbr = GradientBoostingRegressor(**gbr_params)
    
    # Fit the model
    gbr.fit(X_train_std, y_train)
    
    # Print Coefficient of determination R^2
    print("Model Accuracy: %.3f" % gbr.score(X_test_std, y_test))
    
    # Create the mean squared error
    mse = mean_squared_error(y_test, gbr.predict(X_test_std))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
    
    model_file = f"modelfiles/{metric}-{state_code}-model.sav"
    
    pickle.dump(gbr, open(model_file, "wb"))

In [5]:
tmax_df = pd.read_csv('../../data/cleaned_data/tmaxdata.csv',names=["state_code", "date", "value"])
tmin_df = pd.read_csv('../../data/cleaned_data/tmindata.csv',names=["state_code", "date", "value"])
prcp_df = pd.read_csv('../../data/cleaned_data/prcpdata.csv',names=["state_code", "date", "value"])
snow_df = pd.read_csv('../../data/cleaned_data/snowdata.csv',names=["state_code", "date", "value"])

for index, row in states_df.iterrows():
    state_code = row["state_code"]
    print(f"Processing {state_code}")
    buildModel(tmax_df,state_code,"TMAX")
    buildModel(tmin_df,state_code,"TMIN")
    buildModel(prcp_df,state_code,"PRCP")
    buildModel(snow_df,state_code,"SNOW")
    


Processing AL
  state_code        date  value
0         AK  1905-01-01    6.1
1         AK  1905-01-02    7.2
2         AK  1905-01-03    5.0
3         AK  1905-01-04    5.6
4         AK  1905-01-05    5.6


/Users/sanilveeravu/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


Model Accuracy: 0.797
The mean squared error (MSE) on test set: 8.5292
  state_code        date  value
0         AK  1905-01-01  -31.1
1         AK  1905-01-02  -35.6
2         AK  1905-01-03  -34.4
3         AK  1905-01-04  -30.0
4         AK  1905-01-05  -31.1


/Users/sanilveeravu/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


Model Accuracy: 0.819
The mean squared error (MSE) on test set: 14.5852
  state_code        date  value
0         AK  1905-01-01   16.0
1         AK  1905-01-02   11.9
2         AK  1905-01-03   27.9
3         AK  1905-01-04   30.5
4         AK  1905-01-05   19.1


...

Model Accuracy: 0.816
The mean squared error (MSE) on test set: 22.1419
  state_code        date  value
0         AK  1905-01-01   16.0
1         AK  1905-01-02   11.9
2         AK  1905-01-03   27.9
3         AK  1905-01-04   30.5
4         AK  1905-01-05   19.1


/Users/sanilveeravu/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


Model Accuracy: 0.140
The mean squared error (MSE) on test set: 165.1206
  state_code        date  value
0         AK  1905-01-01  305.0
1         AK  1905-01-02  102.0
2         AK  1905-01-03  127.0
3         AK  1905-01-04  267.0
4         AK  1905-01-05  127.0


/Users/sanilveeravu/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


Model Accuracy: 0.256
The mean squared error (MSE) on test set: 7008.7414
